In [ ]:
## fetch stock data for taining from yahoo finance using lstm model
## train the model
## save the model
#!Pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pandas_datareader.data as web
import pickle
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import os 
import pickle
models_path = 'saved_models/'
# fetch stock data from yahoo finance


Prepare Dataset

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.utils import shuffle

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

def prepare_dataset(ticker, from_date, end_date, save_scaler=True):
    # Download stock data with multiple features
    dfs = []

    data = yf.download(ticker, start=from_date, end=end_date)
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Selecting OHLCV columns
        ## append 

    # Append the prepared DataFrame to the list
    dfs.append(data)
    full_data = pd.concat(dfs)
    ## shuffle the data
    #full_data = full_data.sample(frac=1).reset_index(drop=True)
    # Normalize the dat aset
    features_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = features_scaler.fit_transform(full_data)
    
    target = full_data[['Close']]  # Target variable

    # Normalize the target variable separately
    target_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_target = target_scaler.fit_transform(target)

    # Save the scalers for later use
    if save_scaler:
        pickle.dump(features_scaler, open(f'saved_models/{ticker}_features_scaler.pkl', 'wb'))
        pickle.dump(target_scaler, open(f'saved_models/{ticker}_target_scaler.pkl', 'wb'))


    return scaled_data, scaled_target

def create_dataset_multifeature(data, scaled_target, time_step=1, prediction_step=10):
    X, Y = [], []
    for i in range(len(data) - time_step - prediction_step - 1):
        X.append(data[i:(i + time_step), :])
        # Y is now the 'Close' price at 'prediction_step' days ahead
        Y.append(scaled_target[i + time_step + prediction_step - 1, 0])  # 0 index, assuming single column in scaled_target
    return np.array(X), np.array(Y)

def prepare_dataset_for_tickers(tickers, from_date, end_date, save_scaler=True):
    dfs = []
    for ticker in tickers:
        # Download stock data
        data = yf.download(ticker, start=from_date, end=end_date)
        print(data)
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        # data.index = pd.MultiIndex.from_product([[ticker], data.index], names=['Ticker', 'Date'])
        dfs.append(data)
    full_data = pd.concat(dfs)
    print(full_data.head())
    # Normalize features
    features_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = features_scaler.fit_transform(full_data)

    # Extract and normalize target ('Close')
    target = full_data.loc[:, 'Close'].to_frame()
    target_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_target = target_scaler.fit_transform(target)

    if save_scaler:
        pickle.dump(features_scaler, open('saved_models/{ticker}_features_scaler.pkl', 'wb'))
        pickle.dump(target_scaler, open('saved_models/{ticker}_target_scaler.pkl', 'wb'))

    return full_data, scaled_data, scaled_target

def create_sequences_per_stock(data, scaled_target, time_step=1, prediction_step=10):
    X, Y = [], []
    unique_stocks = data.index.get_level_values(0).unique()
    for stock in unique_stocks:
        print(stock)
        stock_data = data.loc[stock]
        stock_scaled_target = scaled_target[data.index.get_level_values(0) == stock]
        
        for i in range(len(stock_data) - time_step - prediction_step + 1):
            X.append(stock_data.iloc[i:(i + time_step)].values)
            Y.append(stock_scaled_target[i + time_step + prediction_step - 1, 0])  # Assuming 'Close' as target
    print(np.array(X).shape, np.array(Y).shape) 
    return shuffle(X, Y, random_state=42)  # np.array(X), np.array(Y)

# Example of how to use these functions:

def prepare_data_for_future_prediction(ticker, time_step, pred_step):
    # Calculate start date with buffer for weekends/holidays and prediction step
    today = datetime.now()
    buffer_days = 2  # Adjust based on typical non-trading days in the market
    start_date_buffer = (time_step + pred_step) * buffer_days
    start_date = (today - timedelta(days=start_date_buffer)).strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    
    # Download the data
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        print(data.tail())
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        print(len(data))
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
    
    # Ensure the scaler used here is the same one used for model training
    try:
        scaler_path = f'saved_models/{ticker}_features_scaler.pkl'
        scaler = pickle.load(open(scaler_path, 'rb'))
    except Exception as e:
        print(f"Error loading scaler: {e}")
        return None
    
    # Scaling the data
    data_scaled = scaler.transform(data)
    X_pred = np.array([data_scaled])
    return X_pred   
def get_model():
        model = Sequential([
        LSTM(256, return_sequences=True, input_shape=(100, 5)),  # 100 time steps, 5 features
        LSTM(128),
        Dense(1)
    ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
        return model



# Assuming X and y datasets as prepared in the earlier step
def train_lstm(model,ticker, X, y, epochs=50):
# Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 5))  # 5 for OHLCV features

    # Define the LSTM model
    # Fit the model
    model.fit(X, y, epochs=epochs, batch_size=32, validation_split=0.2, verbose=2, callbacks=[early_stop])
    #model.save(f'saved_models/{ticker}_lstm_model.keras')
    return model

# Usage example


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
current_date = datetime.now().strftime('%Y-%m-%d')
prev_5_yrs_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
time_step = 100
pred_step = 10
features, targets = prepare_dataset('IZO.CN', prev_5_yrs_date, current_date)
X, y = create_dataset_multifeature(features, targets, time_step, pred_step)
model = get_model()
model = train_lstm(model,'IZO.CN', X, y, 30)




[*********************100%%**********************]  1 of 1 completed

Epoch 1/30



c:\Users\PC\Documents\GitHub\StockPrediction-BioStocks\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 - 5s - 160ms/step - loss: 0.0241 - val_loss: 0.0032
Epoch 2/30
29/29 - 2s - 81ms/step - loss: 0.0098 - val_loss: 0.0014
Epoch 3/30
29/29 - 2s - 82ms/step - loss: 0.0092 - val_loss: 0.0010
Epoch 4/30
29/29 - 2s - 81ms/step - loss: 0.0098 - val_loss: 8.7517e-04
Epoch 5/30
29/29 - 2s - 81ms/step - loss: 0.0092 - val_loss: 9.5667e-04
Epoch 6/30
29/29 - 2s - 82ms/step - loss: 0.0097 - val_loss: 7.0219e-04
Epoch 7/30
29/29 - 2s - 82ms/step - loss: 0.0087 - val_loss: 0.0010
Epoch 8/30
29/29 - 2s - 83ms/step - loss: 0.0086 - val_loss: 7.2246e-04
Epoch 9/30
29/29 - 3s - 91ms/step - loss: 0.0088 - val_loss: 0.0014
Epoch 10/30
29/29 - 3s - 92ms/step - loss: 0.0083 - val_loss: 9.1566e-04
Epoch 11/30
29/29 - 3s - 93ms/step - loss: 0.0081 - val_loss: 0.0014
Epoch 11: early stopping


model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
def train_gradientboosting():
# Flatten X for Gradient Boosting
    X_flat = X.reshape(X.shape[0], -1)

    # Splitting dataset into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

    # Initialize and train the Gradient Boosting Regressor
    gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gbr_model.fit(X_train, y_train)

    # Predictions and evaluate
    y_pred = gbr_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Test MSE: {mse}')
    return gbr_model

In [ ]:
!pip uninstall transformers

In [ ]:

def do_live_prediction(ticker,time_step, pred_step):
    X_pred = prepare_data_for_future_prediction(ticker, time_step, pred_step)
    target_path = f'saved_models\{ticker}_target_scaler.pkl'
    model_path = f'saved_models\{ticker}_lstm_model.keras'
    model = get_model()
    model.load_weights(model_path)
    ## load scaler 
    target_scaler = pickle.load(open(target_path, 'rb'))
    ## load model
    prediction = model.predict(X_pred)
    transformed_pred = target_scaler.inverse_transform(prediction)
    return transformed_pred
y = do_live_prediction("IZO.CN", time_step, pred_step)
print(y)

In [20]:
current_date = datetime.now().strftime('%Y-%m-%d')
yes_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
data = yf.download("IZO.CN", start=yes_date, end=current_date)
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Selecting OHLCV columns
print(data)
        ## append 
ticker_symbol = "IZO.CN"
ticker_data = yf.Ticker(ticker_symbol)
ticker_df = ticker_data.history(period='5d')
print(ticker_df[-1])

[*********************100%%**********************]  1 of 1 completed

            Open  High   Low  Close  Volume
Date                                       
2024-04-03  0.15  0.15  0.15   0.15       0


KeyError: -1

In [7]:
    ticker_symbol = "IZO.CN"
    ticker_data = yf.Ticker(ticker_symbol)
    ticker_df = ticker_data.history(period='1d')
    print(ticker_df)
    current_price = ticker_df['Close'].iloc[-1]


IZO.CN: No price data found, symbol may be delisted (period=1d)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


IndexError: single positional indexer is out-of-bounds

In [ ]:
!pip install streamlit

In [ ]:
train_gradientboosting()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)


# Assuming X and y datasets as prepared in the earlier step
def train_lstm(X, y, ticker, epochs=50):
# Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 5))  # 5 for OHLCV features

    # Define the LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(100, 5)),  # 100 time steps, 5 features
        LSTM(50),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Fit the model
    model.fit(X, y, epochs=epochs, batch_size=32, validation_split=0.2, verbose=2, callbacks=[early_stop])
    model.save(f'saved_models/{ticker}lstm_model.h5')
    return model
ticker = 'ATNM'
model = train_lstm(X, y,ticker,50)


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from tensorflow.keras.models import load_model

pred_step = 30
# Fetch current data
buffer_days = 2  # This is an arbitrary value to increase the buffer for weekends and holidays
end_date = datetime.now()
start_date = end_date - timedelta(days=(time_step + pred_step) * buffer_days)

# Format dates in YYYY-MM-DD format for yfinance
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

# Fetch data
data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# Prepare the latest batch of data for prediction
scaled_data = scaler.transform(data)
time_step = 100  # Assuming using the same time_step as during training
latest_batch = scaled_data[-time_step:].reshape(1, time_step, data.shape[1])  # For LSTM
latest_batch_flat = latest_batch.reshape(1, -1)  # For Gradient Boosting

# Load models (Assuming they are saved and we are loading them for prediction)
# lstm_model = load_model('path_to_saved_lstm_model.h5')
# gbr_model = joblib.load('path_to_saved_gbr_model.pkl')  # Assuming Gradient Boosting model is saved with joblib

# Predict with LSTM
# predicted_price_lstm = lstm_model.predict(latest_batch)
# Scale back the prediction to the original price range if necessary
# predicted_price_lstm = scaler.inverse_transform(predicted_price_lstm)

# Predict with Gradient Boosting Regressor
# predicted_price_gbr = gbr_model.predict(latest_batch_flat)
# Scale back the prediction to the original price range if necessary, depending on how the model was trained
# e.g., predicted_price_gbr = scaler.inverse_transform(predicted_price_gbr.reshape(-1, 1))

# Print predictions (remember to adjust inverse transformations based on your scaling and models)
# print(f"LSTM Predicted Price: {predicted_price_lstm}")
# print(f"Gradient Boosting Predicted Price: {predicted_price_gbr}")


In [ ]:
import os 

def prepare_dataset(dataset, pred_steps=0):
    training_data_len = int(np.ceil( len(dataset) * .95 ))
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    train_data = scaled_data[0:int(training_data_len), :]
    # Split the data into x_train and y_train data sets
    x_train = []
    y_train = []

    for i in range(60, len(train_data) - pred_steps):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i+pred_steps, 0])
        if i<= 61:
            print(x_train)
            print(y_train)
            print()
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    # x_train.shape
    # Create the testing data set
    # Create a new array containing scaled values from index 1543 to 2002 
    test_data = scaled_data[training_data_len - 60: , :]
    # Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
        
    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    # Get the models predicted price values 
    
    return x_train, y_train, x_test, y_test, scaler

def build_train_model(x_train, y_train, ticker, interval,interval_desc, model_name, save_model=True):
    print(x_train.shape[1])
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    ## path to appl stock model
    saving_path = os.path.join(models_path, ticker, interval_desc, model_name +'.h5')
    # model.save(saving_path)
    return model



def prepare_data_train_model(ticker, start_date, end_date, interval,interval_desc,prediction_steps, epochs, batch_size, model_name):
    # Get the data for Apple stock
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    # Print the first 5 rows
    # Create a new dataframe with only the 'Close column 
    data = df.filter(['Close'])
    # Convert the dataframe to a numpy array
    dataset = data.values
    # Get the number of rows to train the model on
    x_train, y_train, x_test, y_test, scaler = prepare_dataset(dataset,prediction_steps)
    print(x_train.shape[1])
    return 
    # save scaler using pickle
    scaler_path = os.path.join(models_path, ticker, interval_desc,'scaler.pkl')
    with open('{}{}\{}\scaler.pkl'.format(models_path,ticker,interval_desc), 'wb') as f:
        pickle.dump(scaler, f)
    
    
    # Build the LSTM model
    model = build_train_model(x_train, y_train, ticker, interval,interval_desc, model_name, save_model=True)
    return model





In [ ]:
## GET CURRENT PRICE OF APPL STOCK USING YAHOO FINANCE
# Get the data for Apple stock
df = yf.download('AAPL', data_source='yahoo', start='2020-01-01', end='2020-03-20')

current_price = df['Close'][-1]

print(current_price)

In [ ]:
    
def get_saved_models(ticker,interval_desc):
    # number of past ticks = 60 
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (60, 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    # Train the model
    model.load_weights('saved_models\{}\{}\stock_prediction.h5'.format(ticker,interval_desc))
    return model


def get_current_price(ticker):
    date_before_60_days = datetime.datetime.now() - datetime.timedelta(days=100)

    df = yf.download(ticker,  start=date_before_60_days, end=datetime.datetime.now() )
    current_price = df['Close'][-1]
    return current_price

appl_model_hr = get_saved_models('AAPL','hr')
appl_model_day = get_saved_models('AAPL','day')
appl_model_week = get_saved_models('AAPL','week')
app_hr_predcitions   = do_prediction(appl_model_hr, 'AAPL',interval='1h',interval_desc="hr")[0][0]
app_day_predcitions  = do_prediction(appl_model_day, 'AAPL',interval='1d',interval_desc="day")[0][0]
app_week_predcitions = do_prediction(appl_model_week, 'AAPL',interval='1d',interval_desc="week")[0][0]
current_aapl_price = get_current_price('AAPL')
amzn_model_hr = get_saved_models('AMZN','hr')
amzn_model_day = get_saved_models('AMZN','day')
amzn_model_week = get_saved_models('AMZN','week')
amzn_hr_predcitions   = do_prediction(appl_model_hr, 'AMZN',interval='1h',interval_desc="hr")[0][0]
amzn_day_predcitions  = do_prediction(appl_model_day, 'AMZN',interval='1d',interval_desc="day")[0][0]
amzn_week_predcitions = do_prediction(appl_model_week, 'AMZN',interval='1d',interval_desc="week")[0][0]
current_amzn_price = get_current_price('AMZN')

print(current_price)

In [ ]:
def do_prediction(model, ticker,interval,interval_desc):
    ## fetch data of the last 60 days only
# Get the stock quote
    date_before_60_days = datetime.datetime.now() - datetime.timedelta(days=100)
    df = yf.download(ticker, start=date_before_60_days, end=datetime.datetime.now(), interval=interval)


    # Print the first 5 rows
    print(df.head())

    # Create a new dataframe with only the 'Close column 
    data = df.filter(['Close'])
    # Convert the dataframe to a numpy array
    dataset = data.values
    len(dataset)
    ## load scalar 
    scaler_path = os.path.join(models_path, ticker, interval_desc,'scaler.pkl')
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    
    scaled_data = scaler.transform(dataset)

    x = scaled_data[-60:]
    x = np.reshape(x, (1, x.shape[0], 1))
    print(x.shape)
    predictions = model.predict(x)
    predictions = scaler.inverse_transform(predictions)
    print(predictions)
    return predictions

In [ ]:
end_date = datetime.datetime.strptime('2022-12-31', '%Y-%m-%d')
start_date = end_date - datetime.timedelta(days=600)
df = yf.download('AAPL', start=start_date, end=end_date, interval='1h')
len(df)

Train model for next hr prediction 


In [ ]:
# date object for '2022-12-31'
## convert string to date 
end_date = datetime.datetime.strptime('2022-12-31', '%Y-%m-%d')
start_date = end_date - datetime.timedelta(days=600)
ticker = 'AAPL'
ticker = 'AMZN'
trained_model = prepare_data_train_model(ticker=ticker,start_date= start_date, end_date=end_date, interval='1h',interval_desc="hr", prediction_steps= 0,epochs= 1, batch_size= 1,model_name= 'stock_prediction')

Train model for next day prediction 

In [ ]:
trained_model = prepare_data_train_model(ticker=ticker,start_date= '2010-01-01', end_date='2022-12-31', interval='1d',interval_desc="day", prediction_steps= 0,epochs= 1, batch_size= 1,model_name= 'stock_prediction')

Train model for next week prediciton

In [ ]:
trained_model = prepare_data_train_model(ticker=ticker,start_date= '2010-01-01', end_date='2022-12-31', interval='1d',interval_desc="week", prediction_steps= 7,epochs= 1, batch_size= 1,model_name= 'stock_prediction')

In [ ]:
trained_model.load_weights('saved_models/AAPL/week/stock_prediction.h5')

In [ ]:
do_prediction(trained_model, ticker='AMZN', interval='1h', interval_desc='hr')
##152.60294
##150.78731
##152.50206

Create training data for day prediction based on the last 60 days.

In [ ]:
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape


In [ ]:
import os
interval = "day"
stock = 'APPL'

path = os.path.join(models_path, 'APPL', interval,'stock_prediction.h5')
print(path)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
interval = "day"
stock = 'APPL'
import os
#path = os.path.join(models_path, 'APPL', interval,'stock_prediction.h5')
#print(path)
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)
## path to appl stock model
saving_path = os.path.join(models_path, stock, interval,'stock_prediction.h5')
model.save(saving_path)

In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values 
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse


In [ ]:
## fetch data of the last 60 days only
# Get the stock quote
date_before_60_days = datetime.datetime.now() - datetime.timedelta(days=100)
df = yf.download('AAPL', start=date_before_60_days, end=datetime.datetime.now(), interval="1d")


# Print the first 5 rows
print(df.head())

# Create a new dataframe with only the 'Close column 
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
len(dataset)
scaler = MinMaxScaler(feature_range=(0,1))
## predict the price of the next day based on the past 60 days
scaled_data = scaler.fit_transform(dataset)

x = scaled_data[-60:]
x = np.reshape(x, (1, x.shape[0], 1))
print(x.shape)
predictions = model.predict(x)
predictions = scaler.inverse_transform(predictions)

In [ ]:
predictions

In [ ]:


df = pd.read_excel('Komplette_Übersicht.xlsx',read_only=True)

In [ ]:
!Pip install openpyxl